## Metrics

In [ ]:
def evaluate_rouge(cands, refs, tokenizer):
  cands_tokenized = tokenize_with_space(cands, tokenizer)
  refs_tokenized = tokenize_with_space(refs, tokenizer)
  r1, r2, rl = cal_rouge_score(refs_tokenized, cands_tokenized)
  # BERTScore = cal_batch_bert_score(cands, refs)
  return r1, r2, rl

def tokenize_with_space(texts, tokenizer):
  output = []
  encoded_texts = tokenizer(texts, max_length=416, truncation=True)
  for text in encoded_texts['input_ids']:
    tokenized_text = " ".join(tokenizer.convert_ids_to_tokens(text, skip_special_tokens=True))
    if(len(tokenized_text)==0):
      output.append("")
      continue
    if(tokenized_text[0]=="▁"): 
      tokenized_text = tokenized_text[1:]
    output.append(tokenized_text.strip())
  return output

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge 
rouge = Rouge()
def cal_rouge_score(hyps, refs, get_average_f1=True):
  '''
  argument: cands, refs [list of string], get_average_f1=True
  return dict of r1, r2, rl score
  if get_average_f1 == True return mean of rouge-1, rouge-2, rouge-L
  '''
  r1 = dict(); r1['precision'] = []; r1['recall'] = []; r1['f1'] = []
  r2 = dict(); r2['precision'] = []; r2['recall'] = []; r2['f1'] = []
  rl = dict(); rl['precision'] = []; rl['recall'] = []; rl['f1'] = []
  for hyp, ref in zip(hyps, refs):
    score = {}
    if(len(hyp)==0 or len(ref)==0):
      score = {
          'rouge-1': {
              'p': 0,
              'r': 0,
              'f': 0
          },
          'rouge-2': {
              'p': 0,
              'r': 0,
              'f': 0
          },
          'rouge-l': {
              'p': 0,
              'r': 0,
              'f': 0
          }
      }
    else: score = rouge.get_scores(hyp, ref)[0]
    r1['precision'].append(score['rouge-1']['p'])
    r1['recall'].append(score['rouge-1']['r'])
    r1['f1'].append(score['rouge-1']['f'])
    
    r2['precision'].append(score['rouge-2']['f'])
    r2['recall'].append(score['rouge-2']['f'])
    r2['f1'].append(score['rouge-2']['f'])

    rl['precision'].append(score['rouge-l']['f'])
    rl['recall'].append(score['rouge-l']['f'])
    rl['f1'].append(score['rouge-l']['f'])
  # score = rouge.get_scores(cands, refs, avg=True)
  # r1['precision_average'] = score['rouge-1']['p']
  # r1['recall_average'] = score['rouge-1']['r']
  # r1['f1_average'] = score['rouge-1']['f']
  
  # r2['precision_average'] = score['rouge-2']['f']
  # r2['recall_average'] = score['rouge-2']['f']
  # r2['f1_average'] = score['rouge-2']['f']

  # rl['precision_average'] = score['rouge-l']['f']
  # rl['recall_average'] = score['rouge-l']['f']
  # rl['f1_average'] = score['rouge-l']['f']
  if(get_average_f1==True): return sum(r1['f1'])/len(r1['f1']), sum(r2['f1'])/len(r2['f1']), sum(rl['f1'])/len(rl['f1'])
  else: return r1, r2, rl

In [ ]:
cands = ['test test test test test test bad']
refs = ['test test']

r1, r2, rl = cal_rouge_score(cands, refs)
print(r1)
print(r2)
print(rl)

0.4444444409876544
0.2857142832653061
0.6666666622222223


In [ ]:
!pip install bert_score==0.3.7
from bert_score import score
import numpy as np

     |████████████████████████████████| 61kB 7.7MB/s 


In [ ]:
import gc

def cal_bert_score(cands, refs, get_average_f1=True):
  '''
  arguments: cands, refs
  return array of presicion, recall, f1, presicion_average, recall_average, f1_average
  if get_average == True return mean of BERTScore
  '''
  p, r, f1 = score(cands, refs, lang="others", verbose=False)
  p = p.numpy()
  r = r.numpy()
  f1 = f1.numpy()
  if(get_average_f1==True): return f1.mean()
  else: return p, r, f1

def cal_batch_bert_score(cands, refs, get_average_f1=True, batch_size=8):
  f1_average = []
  for i in tqdm(range(0,len(cands),batch_size)):
    cand_batch = cands[i:i+batch_size]
    ref_batch = refs[i:i+batch_size]
    res = cal_bert_score(cand_batch, ref_batch)
    f1_average.append(res)
    gc.collect()
  print(f1_average)
  return sum(f1_average)/len(f1_average)

## Segment Sentences

## Oracle A

In [ ]:
!pip -q install torch==1.5.0 torchtext==0.6.0 torchvision==0.6.0
!pip -q install transformers==3.5.0 thai2transformers==0.1.2
!pip install pythainlp

!pip install -Uqq datasets pythainlp==2.2.4 transformers==4.4.0 tensorflow==2.4.0 tensorflow_text emoji seqeval sentencepiece fuzzywuzzy
!npx degit --force https://github.com/vistec-AI/thai2transformers#dev

     |████████████████████████████████| 752.0MB 24kB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 6.6MB 41.4MB/s 
     |████████████████████████████████| 1.2MB 39.1MB/s 
     |████████████████████████████████| 1.3MB 21.6MB/s 
     |████████████████████████████████| 901kB 45.2MB/s 
     |████████████████████████████████| 1.1MB 46.2MB/s 
     |████████████████████████████████| 2.9MB 42.8MB/s 
     |████████████████████████████████| 133kB 59.8MB/s 
     |████████████████████████████████| 11.0MB 58.7MB/s 
     |████████████████████████████████| 225kB 50.3MB/s 
     |████████████████████████████████| 8.7MB 19.0MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 808kB 46.8MB/s 
     |████████████████████████████████| 10.1MB 49.7MB/s 
     |████████████████████████████████| 747kB 41.8MB/s 
     |████████████████████████████████| 245kB 58.7MB/s 
     |████████████████████████████████| 112kB 54.

In [ ]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
from IPython.display import display
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pythainlp.tokenize import word_tokenize, sent_tokenize

from transformers import (
    CamembertTokenizer,
    AutoTokenizer,
    AutoModel,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

from sklearn.utils import shuffle

In [ ]:
from thai2transformers.tokenizers import ThaiWordsNewmmTokenizer
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')

In [ ]:
!gdown --id 1-8IU8qyry-yPXwQ7AXz0GHIgn19QKGZP
!gdown --id 1-J0eqf4ig7cP8bMPRgSFUejshnBFTZoq
!gdown --id 1-IIJFl4AGNr7rRax4YSQTTm7j12YJ0ya

Downloading...
From: https://drive.google.com/uc?id=1-8IU8qyry-yPXwQ7AXz0GHIgn19QKGZP
To: /content/thaisum.csv
2.94GB [00:45, 34.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-J0eqf4ig7cP8bMPRgSFUejshnBFTZoq
To: /content/validation_set.csv
118MB [00:01, 68.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-IIJFl4AGNr7rRax4YSQTTm7j12YJ0ya
To: /content/test_set.csv
119MB [00:02, 55.0MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/thaisum.csv')
val_df = pd.read_csv('/content/drive/MyDrive/validation_set.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test_set.csv')
df = pd.concat([df, val_df, test_df], axis=0)

In [ ]:
data_test = df['body'][369868:380868].tolist()
data_test[:3]

['พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน สมคิด ซัดพูดย้อนแย้งไหลไปไหลมา ลั่น ไม่เคยกลัวในสนามเลือกตั้ง สมชัย กระแทกซ้ำเปิดช่องโกงสนั่นกันทั้งประเทศ ทนายปู มึนรถตู้รับจ้างถูกเล่นงาน ภูมิธรรม ดับกระแส พท.วางตัวผู้นำ ยันพรรคเหนียวแน่นไม่มีปีนเกลียว วิม แจงไม่ได้ล็อกเป้าแค่คน ชินวัตร บิ๊กตู่ รอรายงาน โกตี๋ ตายที่ลาว บิ๊กป้อม โบ้ยศพใครไม่รู้ ผบ.ทบ.ชี้ภาพตกแต่งกันได้ ครม.ตั้ง เลิศวิโรจน์ ขึ้นปลัดเกษตรฯ สุทธิพงษ์ อ.ส่งเสริมปกครองท้องถิ่น ประยุทธ์ รับ รมว.ต่างประเทศสหรัฐฯเยือนไทย ปัดไม่ได้โดนรังเกียจจากนานาชาติ ดอน ยังมั่นใจไปเยือนในปีนี้ แม้เสียงคัดค้านจากฝ่ายการเมืองต่อรูปแบบการเลือกตั้ง ส.ส. สูตรพิสดารของกรรมการร่างรัฐธรรมนูญ (กรธ.) ยังคงดังกระหึ่ม แต่นายมีชัย ฤชุพันธุ์ ประธาน กรธ. ยังคงยืนยันเดินหน้าต่อไป แม้การเลือกตั้งตามรูปแบบใหม

In [ ]:
from tqdm import tqdm
def segment_sent(data):
  ret = []
  for text in tqdm(data):
    ret.append(sent_tokenize(text))
  return ret

In [ ]:
data_segmented_sentence = segment_sent(data_test)

100%|██████████| 11000/11000 [09:15<00:00, 19.80it/s]


In [ ]:
data_segmented_sentence[:3]

[['พอไม่ได้ดังใจก็ร่างใหม่ ',
  'แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ',
  'ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน ',
  'สมคิด ซัดพูดย้อนแย้งไหลไปไหลมา ลั่น ไม่เคยกลัวในสนามเลือกตั้ง สมชัย กระแทกซ้ำเปิดช่องโกงสนั่นกันทั้งประเทศ ทนายปู มึนรถตู้รับจ้างถูกเล่นงาน ภูมิธรรม ดับกระแส พท.วางตัวผู้นำ ยันพรรคเหนียวแน่นไม่มีปีนเกลียว วิม ',
  'แจงไม่ได้ล็อกเป้าแค่คน ชินวัตร บิ๊กตู่ รอรายงาน โกตี๋ ตายที่ลาว บิ๊กป้อม โบ้ยศพใครไม่รู้ ',
  'ผบ.ทบ.ชี้ภาพตกแต่งกันได้ ',
  'ครม.ตั้ง ',
  'เลิศวิโรจน์ ขึ้นปลัดเกษตรฯ สุทธิพงษ์ ',
  'อ.ส่งเสริมปกครองท้องถิ่น ประยุทธ์ รับ รมว.ต่างประเทศสหรัฐฯเยือนไทย ',
  'ปัดไม่ได้โดนรังเกียจจากนานาชาติ ดอน ยังมั่นใจไปเยือนในปีนี้ ',
  'แม้เสียงคัดค้านจากฝ่ายการเมืองต่อรูปแบบการเลือกตั้ง ส.ส. ',
  'สูตรพิสดารของกรรมการร่างรัฐธรรมนูญ (กรธ.) ยังคงดังกระหึ่ม แต่นายมีชัย ฤชุพันธ

In [ ]:
def gold_summary(df, num_train, num_val, num_test):
  return df.iloc[num_train+num_val:num_train+num_val+num_test,:]['summary'].tolist()

In [ ]:
gold_summaries = gold_summary(df, 358868, 11000, 11000)
gold_summaries_sentence_segmented = segment_sent(gold_summaries)

100%|██████████| 11000/11000 [00:27<00:00, 403.21it/s]


In [ ]:
import collections as coll
# stopwords = pkgutil.get_data(__package__, 'smart_common_words.txt')
# stopwords = stopwords.decode('ascii').split('\n')
# stopwords = {key.strip(): 1 for key in stopwords}

def _get_ngrams_count(n, text):
    """Calcualtes n-grams.
    Args:
      n: which n-grams to calculate
      text: An array of tokens
    Returns:
      A set of n-grams
    """
    ngram_dic = coll.defaultdict(int)
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_dic[tuple(text[i:i + n])] += 1
    return ngram_dic

def _get_ngrams(n, text):
    """Calcualtes n-grams.
    Args:
      n: which n-grams to calculate
      text: An array of tokens
    Returns:
      A set of n-grams
    """
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set

def _get_word_ngrams_list(n, text):
    """Calcualtes n-grams.
    Args:
      n: which n-grams to calculate
      text: An array of tokens
    Returns:
      A set of n-grams
    """
    text = sum(text, [])
    ngram_set = []
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.append(tuple(text[i:i + n]))
    return ngram_set

def _get_word_ngrams(n, sentences, do_count=False):
    """Calculates word n-grams for multiple sentences.
    """
    assert len(sentences) > 0
    assert n > 0

    # words = _split_into_words(sentences)

    words = sum(sentences, [])
    # words = [w for w in words if w not in stopwords]
    if do_count:
        return _get_ngrams_count(n, words)
    return _get_ngrams(n, words)
  
def cal_rouge(evaluated_ngrams, reference_ngrams):
    reference_count = len(reference_ngrams)
    evaluated_count = len(evaluated_ngrams)

    overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
    overlapping_count = len(overlapping_ngrams)

    if evaluated_count == 0:
        precision = 0.0
    else:
        precision = overlapping_count / evaluated_count

    if reference_count == 0:
        recall = 0.0
    else:
        recall = overlapping_count / reference_count

    f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))
    return {"f": f1_score, "p": precision, "r": recall}

def greedy_selection(doc_sent_list, abstract_sent_list, summary_size):
  max_rouge = 0.0
  abstract = sum(abstract_sent_list, [])
  abstract = ' '.join(abstract).split()
  sents = [' '.join(s).split() for s in doc_sent_list]
  evaluated_1grams = [_get_word_ngrams(1, [sent]) for sent in sents]
  reference_1grams = _get_word_ngrams(1, [abstract])
  evaluated_2grams = [_get_word_ngrams(2, [sent]) for sent in sents]
  reference_2grams = _get_word_ngrams(2, [abstract])

  selected = []
  for s in range(summary_size):
      cur_max_rouge = max_rouge
      cur_id = -1
      for i in range(len(sents)):
          if (i in selected):
              continue
          c = selected + [i]
          candidates_1 = [evaluated_1grams[idx] for idx in c]
          candidates_1 = set.union(*map(set, candidates_1))
          candidates_2 = [evaluated_2grams[idx] for idx in c]
          candidates_2 = set.union(*map(set, candidates_2))
          rouge_1 = cal_rouge(candidates_1, reference_1grams)['f']
          rouge_2 = cal_rouge(candidates_2, reference_2grams)['f']
          rouge_score = rouge_1 + rouge_2
          if rouge_score > cur_max_rouge:
              cur_max_rouge = rouge_score
              cur_id = i
      if (cur_id == -1):
          return selected
      selected.append(cur_id)
      max_rouge = cur_max_rouge

  return sorted(selected)

In [ ]:
def oracle_a(paragraph_raw, gold_summaries):
  ret = []
  sent_selected = []
  for docs, abs in tqdm(zip(paragraph_raw, gold_summaries)):
    t_docs = [[x] for x in docs]
    t_abs = [[x] for x in abs]
    selected = greedy_selection(t_docs, t_abs, 4)
    new_docs = []
    sent_selected.append(selected)
    for idx in selected:
      new_docs.append(docs[idx])
    ret.append("".join(new_docs))
  return ret, sent_selected

In [ ]:
data_segmented_sentence[0]

['พอไม่ได้ดังใจก็ร่างใหม่ ',
 'แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ',
 'ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน ',
 'สมคิด ซัดพูดย้อนแย้งไหลไปไหลมา ลั่น ไม่เคยกลัวในสนามเลือกตั้ง สมชัย กระแทกซ้ำเปิดช่องโกงสนั่นกันทั้งประเทศ ทนายปู มึนรถตู้รับจ้างถูกเล่นงาน ภูมิธรรม ดับกระแส พท.วางตัวผู้นำ ยันพรรคเหนียวแน่นไม่มีปีนเกลียว วิม ',
 'แจงไม่ได้ล็อกเป้าแค่คน ชินวัตร บิ๊กตู่ รอรายงาน โกตี๋ ตายที่ลาว บิ๊กป้อม โบ้ยศพใครไม่รู้ ',
 'ผบ.ทบ.ชี้ภาพตกแต่งกันได้ ',
 'ครม.ตั้ง ',
 'เลิศวิโรจน์ ขึ้นปลัดเกษตรฯ สุทธิพงษ์ ',
 'อ.ส่งเสริมปกครองท้องถิ่น ประยุทธ์ รับ รมว.ต่างประเทศสหรัฐฯเยือนไทย ',
 'ปัดไม่ได้โดนรังเกียจจากนานาชาติ ดอน ยังมั่นใจไปเยือนในปีนี้ ',
 'แม้เสียงคัดค้านจากฝ่ายการเมืองต่อรูปแบบการเลือกตั้ง ส.ส. ',
 'สูตรพิสดารของกรรมการร่างรัฐธรรมนูญ (กรธ.) ยังคงดังกระหึ่ม แต่นายมีชัย ฤชุพันธุ์ ',
 'ประธ

In [ ]:
summaries_oracle_a = oracle_a(data_segmented_sentence, gold_summaries_sentence_segmented)

11000it [00:11, 941.34it/s]


In [ ]:
# summaries_oracle_a[0][:10]
text_summaries_oracle_a = summaries_oracle_a[0]
text_summaries_oracle_a[:5]

['ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ',
 'เมินประเด็นจุดโทษปัญหาในเกมพรีเมียร์ลีก อังกฤษ นัดที่แตนอาละวาดเปิดบ้านพ่าย คริสตัล พาเลซ 0-1ชี้ทีมของเขาเล่นไม่ดีพอเอง กีเก ซานเชซ ฟลอเรส\xa0 กุนซือเลือดกระทิงของทีมวัตฟอร์ด\xa0 ',
 'เบื้องต้นพบผู้เสียชีวิตแล้ว 30 ศพ บาดเจ็บ 400 ราย ระบบสาธารณูปโภคพังพินาศ ทางการเร่งระดมเจ้าหน้าที่ตำรวจ เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร้างความเสียหายหลายหมู่บ้าน ',
 'ระหว่าง 17-18 พ.ค. ที่ห้องอาซัวร์ เรเนซองส์ เป็นศูนย์รวมของความคิดสร้างสรรค์และคอมมูนิตี้คนรุ่นใหม่ ล่าสุด อุสรา ยงปิยะกุล คว้าศิลปินไทยรุ่นใหม่จากนิวยอร์ก กันตภณ เมธีกุล มาร่วมคอลลาโบเรชั่น สินค้าลิมิเต็ด อิดิชั่น ODS X GONGKAN ให้คนรักงานศิลป์คนไทย ริเริ่มจัดบุฟเฟต์ทุเรียนจนเป็นเทรนด์นิยม ปีนี้ ',
 'บ้านตะวันลับ Twilight Villa ถูกจัดตั้งขึ้นในปี

In [ ]:
gold_summaries[0]

'มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง'

In [ ]:
r1, r2, rl = evaluate_rouge(text_summaries_oracle_a, gold_summaries, tokenizer)
print(r1, r2, rl)

0.7808340934426083 0.7448053091355908 0.76082355242242


In [ ]:
BERTScore = cal_batch_bert_score(text_summaries_oracle_a, gold_summaries, batch_size=128)

100%|██████████| 86/86 [14:12<00:00,  9.91s/it]

[0.89319736, 0.8836837, 0.88032484, 0.8779398, 0.8858798, 0.8809267, 0.88100886, 0.87674797, 0.89173806, 0.87507653, 0.8732444, 0.88800573, 0.8811226, 0.8898343, 0.8793214, 0.8658464, 0.8761872, 0.8603794, 0.88456786, 0.88599443, 0.8757633, 0.89016646, 0.89399076, 0.87415874, 0.87273276, 0.880453, 0.8796413, 0.8908386, 0.8764343, 0.87796783, 0.88312936, 0.8836874, 0.8804669, 0.8573898, 0.87110853, 0.879704, 0.8681684, 0.890043, 0.8747535, 0.890775, 0.88171005, 0.87800777, 0.89452434, 0.8808714, 0.88298535, 0.8807734, 0.88243896, 0.8797245, 0.88437307, 0.88141817, 0.8905538, 0.88761854, 0.8906372, 0.88758254, 0.8784401, 0.87384605, 0.8899583, 0.8804734, 0.89921236, 0.88413066, 0.890375, 0.87512314, 0.87130505, 0.88755757, 0.8972032, 0.8746815, 0.8713199, 0.8795327, 0.8980079, 0.8765938, 0.8822874, 0.8898929, 0.89170563, 0.8693235, 0.88157827, 0.8730935, 0.882172, 0.8828261, 0.8888686, 0.8984972, 0.89771914, 0.89279693, 0.87961847, 0.87243587, 0.88805604, 0.8856912]


In [ ]:
print(BERTScore)

0.8822086370268534


In [ ]:
np.save('text_summaries_oracle_a', np.array(text_summaries_oracle_a))

## Lead-3

In [ ]:
def lead_3(data):
  ret = []
  for text in tqdm(data):
    ret.append("".join(text[:3]))
  return ret

In [ ]:
summaries_lead_3 = lead_3(data_segmented_sentence)

100%|██████████| 11000/11000 [00:00<00:00, 283780.66it/s]


In [ ]:
summaries_lead_3[:3]

['พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน ',
 'กีเก ซานเชซ ฟลอเรส\xa0 กุนซือเลือดกระทิงของทีมวัตฟอร์ด\xa0 เมินประเด็นจุดโทษปัญหาในเกมพรีเมียร์ลีก อังกฤษ นัดที่แตนอาละวาดเปิดบ้านพ่าย คริสตัล พาเลซ 0-1ชี้ทีมของเขาเล่นไม่ดีพอเอง สำนักข่าวต่างประเทศรายงานวันที่ 27 ก.ย. ว่า กีเก ซานเชซ ฟลอเรส\xa0 ผู้จัดการทีมชาวสเปน ของ แตนอาละวาด วัตฟอร์ด\xa0 ยอมรับทีมของเขาเล่นได้ไม่ดีพอเอง ',
 'ตาย30-เจ็บ400 เส้นทางตัดขาด เข้าช่วยลำบาก เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร้างความเสียหายหลายหมู่บ้าน เบื้องต้นพบผู้เสียชีวิตแล้ว 30 ศพ บาดเจ็บ 400 ราย ระบบสาธารณูปโภคพังพินาศ ทางการเร่งระดมเจ้าหน้าที่ตำรวจ ']

In [ ]:
gold_summaries[:3]

['มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง',
 'กีเก ซานเชซ ฟลอเรส  กุนซือเลือดกระทิงของทีมวัตฟอร์ด  เมินประเด็นจุดโทษปัญหาในเกมพรีเมียร์ลีก อังกฤษ นัดที่แตนอาละวาดเปิดบ้านพ่าย คริสตัล พาเลซ 0-1ชี้ทีมของเขาเล่นไม่ดีพอเอง',
 'เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร้างความเสียหายหลายหมู่บ้าน เบื้องต้นพบผู้เสียชีวิตแล้ว 30 ศพ บาดเจ็บ 400 ราย ระบบสาธารณูปโภคพังพินาศ']

In [ ]:
r1, r2, rl = evaluate_rouge(summaries_lead_3, gold_summaries, tokenizer)
print(r1, r2, rl)

0.604528036727442 0.5596666594573724 0.6195494182077482


In [ ]:
BERTScore = cal_batch_bert_score(summaries_lead_3, gold_summaries, batch_size=128)

100%|██████████| 86/86 [15:44<00:00, 10.99s/it]

[0.8345444, 0.8464756, 0.84273994, 0.8378253, 0.8387501, 0.8473702, 0.8320834, 0.83725685, 0.8413744, 0.8321419, 0.8419398, 0.84918404, 0.8347242, 0.8334775, 0.8336364, 0.832308, 0.8470652, 0.83865845, 0.8404116, 0.8402912, 0.8413036, 0.8509978, 0.84434867, 0.8459867, 0.8289015, 0.8424605, 0.8345797, 0.8530023, 0.8438095, 0.8451314, 0.83553374, 0.8405249, 0.83000356, 0.827894, 0.8329407, 0.8314443, 0.83256584, 0.8395792, 0.84691614, 0.8428999, 0.83579373, 0.8395871, 0.84907603, 0.8470104, 0.8377347, 0.8397471, 0.82915545, 0.83986306, 0.8349063, 0.84085053, 0.8430065, 0.83317757, 0.84134436, 0.83819497, 0.83277583, 0.8472707, 0.8360435, 0.83738154, 0.854439, 0.83460546, 0.8452349, 0.8466543, 0.83415437, 0.8331202, 0.8566128, 0.8388778, 0.83837813, 0.8298112, 0.8465066, 0.83458006, 0.8427532, 0.83417904, 0.8561456, 0.82710695, 0.84181404, 0.8374928, 0.8336501, 0.8468715, 0.849596, 0.8510137, 0.8446379, 0.84719455, 0.8470298, 0.83733153, 0.8330279, 0.82907903]


In [ ]:
BERTScore

0.8398593068122864

In [ ]:
np.save('summaries_lead_3', np.array(summaries_lead_3))